In [1]:
import sys
import os
import shutil

In [2]:
sys.path.append('..')

In [3]:
from jamo import j2hcj, h2j
import pandas as pd
import numpy as np
from konlpy.tag import Twitter

from sklearn.metrics import accuracy_score, f1_score
twitter = Twitter()

/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [4]:
from utils.prepare_data import *
import time
from utils.model_helper import *

/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
from custum_dataset import digi_data_load, testcase_add_data_load, testcase_shuffle_data_load
from attn_bi_lstm import Shared_ABLSTM

In [6]:
_, _, _, _, cols = digi_data_load()

4244 1062


In [7]:
onehot_mat = pd.get_dummies(cols).astype(float)
onehot_mat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 33 columns):
DIGITAL_Pa/s문의          33 non-null float64
DIGITAL_P가격문의           33 non-null float64
DIGITAL_P결제문의           33 non-null float64
DIGITAL_P교환문의           33 non-null float64
DIGITAL_P기타faq문의        33 non-null float64
DIGITAL_P대화종료           33 non-null float64
DIGITAL_P도움말보기          33 non-null float64
DIGITAL_P반품문의           33 non-null float64
DIGITAL_P배송문의           33 non-null float64
DIGITAL_P배송일문의          33 non-null float64
DIGITAL_P배송지변경문의        33 non-null float64
DIGITAL_P본인인증문의         33 non-null float64
DIGITAL_P상담시간           33 non-null float64
DIGITAL_P상담요청           33 non-null float64
DIGITAL_P상담원연결          33 non-null float64
DIGITAL_P상담품목           33 non-null float64
DIGITAL_P상품문의_P상담원연결    33 non-null float64
DIGITAL_P전화요청           33 non-null float64
DIGITAL_P주문이력확인         33 non-null float64
DIGITAL_P주문취소           33 non-null float64
DIGITAL_P차이점     

In [8]:
df = pd.read_csv('../../nlu-model-analysis/binary_dataset.csv',sep='\t',index_col='Unnamed: 0')
df = df.sample(len(df),random_state=10)
df['binary_class'].value_counts()

0    60407
1    47390
Name: binary_class, dtype: int64

In [9]:
def class2onehot(label):
    if label not in onehot_mat.columns.tolist():
        return np.zeros((len(onehot_mat),))
    
    return onehot_mat[label].values.reshape(-1,)

In [10]:
train_df = df.iloc[:int(len(df)*0.8)]
test_df = df.iloc[int(len(df)*0.8):]

In [11]:
TRAIN_DOC, TRAIN_LABEL, TRAIN_BIN_LABEL, TEST_DOC, TEST_LABEL, TEST_BIN_LABEL = \
    train_df['sent'], train_df['class'], train_df['binary_class'], \
    test_df['sent'], test_df['class'], test_df['binary_class']

In [12]:
train_intent = np.asarray(TRAIN_LABEL.apply(lambda x: class2onehot(x).tolist()).tolist())
test_intent = np.asarray(TEST_LABEL.apply(lambda x: class2onehot(x).tolist()).tolist())

In [13]:
print(train_intent.shape, test_intent.shape)

(86237, 33) (21560, 33)


In [14]:
dis_train = TRAIN_BIN_LABEL
dis_test = TEST_BIN_LABEL

In [15]:
%%time
train_doc = TRAIN_DOC.tolist()
test_doc = TEST_DOC.tolist()

#train_doc = pd.Series(list(map(lambda x: ' '.join(twitter.morphs(x)), train_doc)))
#test_doc = pd.Series(list(map(lambda x: ' '.join(twitter.morphs(x)), test_doc)))
train_doc = pd.Series(list(map(lambda x: ' '.join(list(x.replace(' ','ⓢ'))), train_doc)))
test_doc = pd.Series(list(map(lambda x: ' '.join(list(x.replace(' ','ⓢ'))), test_doc)))

CPU times: user 206 ms, sys: 7.27 ms, total: 214 ms
Wall time: 212 ms


In [16]:
y_train = train_intent
y_test = test_intent

In [17]:
x_train, x_test, tokenizer = \
        data_preprocessing_v2(train_doc, test_doc, max_len=64)
vocab_size = tokenizer.num_words

In [18]:
tokenizer.word_docs

{'스': 6841,
 '디': 2779,
 '이': 18457,
 '레': 1484,
 '플': 878,
 '우': 1820,
 '로': 6714,
 '렐': 20,
 '코': 950,
 '쿠': 3220,
 '나': 8175,
 '요': 22137,
 '폰': 3697,
 'ⓢ': 52623,
 '안': 5265,
 '되': 4566,
 '헤': 262,
 '장': 3556,
 '식': 907,
 '어': 12477,
 '증': 1099,
 '영': 539,
 '발': 1000,
 '해': 6880,
 '행': 774,
 '현': 539,
 '수': 4232,
 '금': 2371,
 '필': 751,
 'h': 1236,
 '3': 2718,
 'b': 904,
 '6': 1897,
 '0': 5287,
 '8': 1942,
 '겠': 576,
 '다': 8135,
 '담': 1356,
 '하': 9381,
 '습': 1773,
 '니': 7587,
 '에': 6687,
 '세': 4819,
 '주': 6737,
 '알': 1957,
 '려': 3244,
 '약': 482,
 '치': 2188,
 '아': 8703,
 '닥': 95,
 '가': 12256,
 '동': 1412,
 '방': 1993,
 '탈': 254,
 '것': 1040,
 '는': 11737,
 '여': 3017,
 '뭐': 1242,
 '곳': 169,
 '기': 8156,
 'n': 2668,
 '줄': 296,
 '무': 1994,
 '고': 10140,
 '씨': 321,
 'ㅓ': 70,
 '오': 3938,
 '옹': 42,
 '제': 7681,
 '결': 2078,
 '성': 1935,
 '청': 2719,
 '손': 165,
 '톱': 26,
 '양': 626,
 '환': 2425,
 '떻': 2150,
 '게': 4486,
 '은': 5570,
 '교': 744,
 '르': 692,
 '구': 5439,
 '트': 6072,
 '유': 1803,
 '위': 522,
 '시

In [19]:
config = {
    "max_len": 64,
    "hidden_size": 64,
    "vocab_size": vocab_size,
    "embedding_size": 128,
    "n_class": 33,
    "learning_rate": 0.00012, #1e-3,
    "decay": 0.90,
    "batch_size": 512,
    "train_epoch": 20,
    "dis_class": 2
}

In [20]:
modelname = 'shared_binary_classifier'

try:
    shutil.rmtree('./output/'+modelname)
except OSError as e:
    if e.errno == 2:
        # 파일이나 디렉토리가 없음!
        print ('No such file or directory to remove')
        pass
    else:
        raise

No such file or directory to remove


In [21]:
sess = tf.Session()

classifier = Shared_ABLSTM(sess, config)
classifier.build_graph(modelname)

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

resume = False
if resume:
    saver.restore(sess, "./Saver/{}/{}.ckpt".format(modelname, modelname))

start = time.time()
step = 0
lr = config['learning_rate']

building graph
graph built successfully!


In [ ]:
for e in range(config["train_epoch"]):
    
    t0 = time.time()
    print("Epoch %d start !" % (e + 1))
    for x_batch, y_batch, dis_batch in shared_fill_feed_dict(x_train, y_train, dis_train, config["batch_size"]):
        _, global_step = sess.run([classifier.train_op, classifier.global_step],
                feed_dict={classifier.x:x_batch,
                           classifier.label:y_batch,
                           classifier.dis_label:dis_batch,
                           classifier.keep_prob:0.5,
                           classifier.LEARNING_RATE: lr})
    
        if step % 10 == 0:
            train_loss, act_train_loss, dis_train_loss, dis_acc = sess.run([classifier.loss, 
                                                                   classifier.n_loss,
                                                                   classifier.dis_loss,
                                                                   classifier.dis_accuracy],
                                                                             feed_dict={
                                                                                classifier.x:x_batch,
                                                                                classifier.label:y_batch,
                                                                                classifier.dis_label:dis_batch,
                                                                                classifier.keep_prob:1})
            print(train_loss, act_train_loss, dis_train_loss, dis_acc)
            
#             feed_dict = make_test_feed_dict(classifier, (x_batch, y_batch))
#             tr_merged = sess.run(classifier.merge, feed_dict)
#             feed_dict = make_test_feed_dict(classifier, (x_test, y_test))
#             ts_merged = sess.run(classifier.merge, feed_dict)
#             classifier.train_writer.add_summary(tr_merged, step)
#             classifier.test_writer.add_summary(ts_merged, step)
            
        step += 1
    t1 = time.time()
    if e+1 % 10: 
        lr = lr*config['decay']
#     print("Train Epoch time:  %.3f s" % (t1 - t0))
#     dev_acc = run_eval_step(classifier, sess, dev_batch)
#     print("validation accuracy: %.3f " % dev_acc)

Epoch 1 start !
4.1676407 3.4502873 0.7173536 0.4765625
4.1177683 3.4308863 0.68688214 0.6484375
4.0453334 3.3827467 0.6625865 0.65625
4.045138 3.3911107 0.65402734 0.6542969
3.9914255 3.3456483 0.6457773 0.6777344
3.975822 3.345126 0.63069594 0.69921875
3.9110274 3.2971563 0.6138712 0.703125
3.834674 3.2092872 0.6253868 0.6777344
3.8651302 3.2705107 0.5946196 0.734375
3.8414829 3.2341955 0.6072874 0.703125
3.792411 3.1926005 0.5998106 0.71484375
3.8741393 3.2596116 0.6145276 0.6816406
3.8386545 3.2538745 0.58477986 0.72265625
3.840662 3.248107 0.5925549 0.6972656
3.7215934 3.1595578 0.56203556 0.734375
3.7747412 3.204884 0.56985706 0.7285156
3.7411845 3.1818216 0.5593629 0.7246094
Epoch 2 start !
3.8297806 3.26835 0.5614307 0.7285156
3.7456086 3.1965013 0.5491073 0.73046875
3.7813845 3.2325525 0.5488319 0.73828125
3.74139 3.1846726 0.5567174 0.7207031
3.7580004 3.215508 0.5424924 0.74609375
3.747508 3.1628075 0.5847006 0.69921875
3.6750321 3.140533 0.53449905 0.7558594
3.702932 3.1763

In [17]:
if "Saver" not in os.listdir("./"):
    os.makedirs("./Saver")
if modelname not in os.listdir("./Saver"):
    os.makedirs("./Saver/{}".format(modelname))
saver.save(sess, "./Saver/{}/{}.ckpt".format(modelname, modelname))

'./Saver/binary_classifier/binary_classifier.ckpt'

In [18]:
print("Training finished, time consumed : ", time.time() - start, " s")
print("Start evaluating:  \n")
cnt = 0
test_acc = 0
for x_batch, y_batch in fill_feed_dict(x_test, y_test, config["batch_size"]):
    acc = run_eval_step(classifier, sess, (x_batch, y_batch))
    test_acc += acc
    cnt += 1

print("Test accuracy : %f %%" % (test_acc / cnt * 100))

Training finished, time consumed :  5620.205155849457  s
Start evaluating:  

Test accuracy : 87.662760 %
